# Imports

In [8]:
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion
from pathlib import Path
import subprocess


# Parse Input

In [9]:
class InputData:
    def __init__(self, input_text: Path):
        self.parse(input_text)
        self.full_description = f"{self.signature}\n{self.description}"
    
    def __repr__(self) -> str:
        return f"function_name: {self.function_name}\nsignature: {self.signature}\ndescription: {self.description}\nexamples: {self.examples}\ntemperatures: {self.temperatures}\nnum: {self.num}\nunit_tests: {self.unit_tests}"
    
    def parse(self, input_text: Path):
        with open(input_text, 'r') as f:
            self.text = f.readlines()
        self.text = [line.strip().removesuffix(";") for line in self.text if line.strip() != '']
        self.function_name = self.text[0]
        self.description = ""
        for i, line in enumerate(self.text[2:]):
            if '"""' in line:
                break
            if 'signature' in line:
                self.signature = line
            else:
                self.description += f"{line} "
        
        next_part = self.text[i + 1:]
        self.examples = []
        for i, line in enumerate(next_part):
            if "example:" in line:
                self.examples.append(line)
            if "temperatures:" in line:
                self.temperatures = line.split(":")[1].split(",")
                self.temperatures = [float(temp) for temp in self.temperatures]
            if "num" in line:
                self.num = int(line.split('=')[1])
            if "unit_tests" in line:
                self.unit_tests = next_part[i + 1:]
                break

In [10]:
input_text = Path('input.txt')
input_data = InputData(input_text)

# Intigrate model

In [11]:
AZURE_OPENAI_API_KEY="e865ffe4387f416ea57d7394ee09fb2e"
AZURE_OPENAI_ENDPOINT="https://openaifor3267.openai.azure.com/"
AZURE_OPENAI_API_VERSION = "2023-12-01-preview"
model = "gpt-35-16k"

In [13]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT
)

In [14]:
def sent_prompt(prompt: str, temp: float, num_of_samples: int) -> ChatCompletion:
    response = client.chat.completions.create(
        model=model,
        temperature=temp,
        n=num_of_samples,
        max_tokens=100,
        messages=[
            {"role": "system", "content": """Act as experienced python developer,
            and provide the implementation for the following functions, make sure to only provide the implementation for the function and not
            put any redundant text as comment, the function should compile without any interventions."""},
            {"role": "user", "content": prompt}
        ]
    )
    return response

In [15]:
responses = []
for temp in input_data.temperatures:
    response_without_ex = {}
    response = sent_prompt(input_data.full_description, temp, input_data.num)
    response_without_ex["temp"] = temp
    response_without_ex["isExample"] = False
    response_without_ex["response"] = response
    responses.append(response_without_ex)

    response_with_ex = {}
    prompt = input_data.full_description + "\n" + "\n".join(input_data.examples)
    response = sent_prompt(prompt, temp, input_data.num)
    response_with_ex["temp"] = temp
    response_with_ex["isExample"] = True
    response_with_ex["response"] = response
    responses.append(response_with_ex)

# Generate Files

In [31]:
file_names = []
for result in responses:
    for i, choice in enumerate(result['response'].choices):
        file_name = f"{input_data.function_name}-{'ex-' if result['isExample'] else ''}s{i + 1}-t{int(result['temp'] * 10)}.py"
        unittest = [f"print({test.replace('->', '==')})" for test in input_data.unit_tests]
        unittest_str = '\n'.join(unittest)
        file_content = f"{choice.message.content}\n\n{unittest_str}"

        with open(file_name, 'w') as f:
            f.write(file_content)
        print(f"File {file_name} created")
        file_names.append(file_name)

File sqrt_list-s1-t0.py created
File sqrt_list-s2-t0.py created
File sqrt_list-s3-t0.py created
File sqrt_list-ex-s1-t0.py created
File sqrt_list-ex-s2-t0.py created
File sqrt_list-ex-s3-t0.py created
File sqrt_list-s1-t8.py created
File sqrt_list-s2-t8.py created
File sqrt_list-s3-t8.py created
File sqrt_list-ex-s1-t8.py created
File sqrt_list-ex-s2-t8.py created
File sqrt_list-ex-s3-t8.py created


# Run Tests and gather results

In [32]:
template_results = """
passed_without_examples={passed_without_examples}
failed_without_examples={failed_without_examples}
average_without_examples={average_without_examples}

passed_with_examples={passed_with_examples}
failed_with_examples={failed_with_examples}
average_with_examples={average_with_examples}
"""
stat_file_name = f"{input_data.function_name}-stats.txt"
t_to_stats = {}

for t in input_data.temperatures:
    t_to_stats[t] = {
        "passed_with_ex": 0,
        "failed_with_ex": 0,
        "passed_no_ex": 0,
        "failed_no_ex": 0,
    }
    for f_name in file_names:
        if f"t{int(t*10)}" in f_name:
            print(f"Running file: {f_name}")
            result = subprocess.run(['python3', f_name], capture_output=True, text=True)
            if result.returncode:
                failed_tests = len(input_data.unit_tests)
                pass_tests = 0
            else:
                pass_tests, failed_tests = result.stdout.count("True"), result.stdout.count("False")
            if "ex" in f_name:
                t_to_stats[t]['passed_with_ex'] += pass_tests
                t_to_stats[t]['failed_with_ex'] += failed_tests
            else:
                t_to_stats[t]['passed_no_ex'] += pass_tests
                t_to_stats[t]['failed_no_ex'] += failed_tests                 

file_content = f"LLM={model}\n"

for t, stat in t_to_stats.items():
    file_content += f"temperature={t}"
    file_content += template_results.format(passed_without_examples=stat['passed_no_ex'],
                            failed_without_examples=stat['failed_no_ex'],
                            average_without_examples=stat['passed_no_ex']/(stat['passed_no_ex']+stat['failed_no_ex']),
                            passed_with_examples=stat['passed_with_ex'],
                            failed_with_examples=stat['failed_with_ex'],
                            average_with_examples=stat['passed_with_ex']/(stat['passed_with_ex']+stat['failed_with_ex']),)
    file_content += "\n\n"
     

with open(stat_file_name, 'w') as f:
        f.write(file_content)

print(f'File {stat_file_name} created.')

Running file: sqrt_list-s1-t0.py
Running file: sqrt_list-s2-t0.py
Running file: sqrt_list-s3-t0.py
Running file: sqrt_list-ex-s1-t0.py
Running file: sqrt_list-ex-s2-t0.py
Running file: sqrt_list-ex-s3-t0.py
Running file: sqrt_list-s1-t8.py
Running file: sqrt_list-s2-t8.py
Running file: sqrt_list-s3-t8.py
Running file: sqrt_list-ex-s1-t8.py
Running file: sqrt_list-ex-s2-t8.py
Running file: sqrt_list-ex-s3-t8.py
File sqrt_list-stats.txt created.
